# Chaining search


## UI initialization

In [13]:
import ipywidgets as widgets
from IPython.display import display
# UI elements for query
queryLabel = widgets.HTML(value="<b>Query:</b>")
queryField = widgets.Text()
queryHbox = widgets.HBox([queryLabel, queryField])

# UI elements for corpus
corpusLabel = widgets.HTML(value="<b>Corpus:</b>")
corpusField = widgets.Dropdown(
    options=['1', '2', '3'],
    value='2',
    description='Number:',
    disabled=False,
)
corpusHbox = widgets.HBox([corpusLabel, corpusField])


display(queryHbox)
display(corpusHbox)

## Corpus search
 * Place the query you would like to search for in the QUERY field.
 * Place the corpus you would like to search in in the CORPUS field.

In [4]:
# TODO: replace this with Jupyter widgets (GUI input elements)
QUERY = r'[lemma="boek"]'
CORPUS = "opensonar" # One of: opensonar, zeebrieven, gysseling, chn, nederlab

### DO NOT EDIT BELOW THIS LINE
import requests
import pandas
import xml.etree.ElementTree as ET

def parse_xml(text):
    print("Hallo hallo")

# Do request to federated content search corpora, so we get same output format for every corpus
url = "http://portal.clarin.inl.nl/fcscorpora/clariah-fcs-endpoints/sru?operation=searchRetrieve&queryType=fcs&x-fcs-context=" + CORPUS + "&maximumRecords=20&query=" + QUERY;
response = requests.get(url)
response_text = response.text
parse_xml(response_text)

Hallo hallo


## Lexicon search